In [ ]:
# !pip install statsmodels

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import math
import numpy as np 
import pandas as pd 
import polars as pl 
import matplotlib.pyplot as plt 
import seaborn as sns
import holidays
import lightgbm as lgb
import tensorflow as tf, deeptables as dt
from tensorflow.keras.utils import plot_model
from tensorflow_addons.optimizers import AdamW
from tensorflow.python.keras import backend as K
from deeptables.models import DeepTable, ModelConfig
from deeptables.models import deepnets
import joblib
from datetime import timedelta

print('Tensorflow version:', tf.__version__)
print('DeepTables version:', dt.__version__)

# fast ai libraries
from fastai.tabular.all import *

# constants
SEED = 2024 # global seed for notebook
BATCH_SIZE = 1024
EPOCHS = 20

#library from Yelim
from statsmodels.tsa.stattools import acf, pacf
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import math
import numpy as np 
import pandas as pd 
import polars as pl 
import matplotlib.pyplot as plt 
import holidays


from datetime import timedelta

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#library from Yelim
from statsmodels.tsa.stattools import acf, pacf
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
class DataStorage:
    """
    This class was copied out from:
    https://www.kaggle.com/code/vitalykudelya/enefit-object-oriented-gbdt
    """
    
    root = "/kaggle/input/predict-energy-behavior-of-prosumers"

    data_cols = [
        "target",
        "county",
        "is_business",
        "product_type",
        "is_consumption",
        "datetime",
        "row_id",
        "prediction_unit_id",
    ]
    client_cols = [
        "product_type",
        "county",
        "eic_count",
        "installed_capacity",
        "is_business",
        "date",
    ]
    gas_prices_cols = ["forecast_date", "lowest_price_per_mwh", "highest_price_per_mwh"]
    electricity_prices_cols = ["forecast_date", "euros_per_mwh"]
    forecast_weather_cols = [
        "latitude",
        "longitude",
        "hours_ahead",
        "temperature",
        "dewpoint",
        "cloudcover_high",
        "cloudcover_low",
        "cloudcover_mid",
        "cloudcover_total",
        "10_metre_u_wind_component",
        "10_metre_v_wind_component",
        "forecast_datetime",
        "direct_solar_radiation",
        "surface_solar_radiation_downwards",
        "snowfall",
        "total_precipitation",
    ]
    historical_weather_cols = [
        "datetime",
        "temperature",
        "dewpoint",
        "rain",
        "snowfall",
        "surface_pressure",
        "cloudcover_total",
        "cloudcover_low",
        "cloudcover_mid",
        "cloudcover_high",
        "windspeed_10m",
        "winddirection_10m",
        "shortwave_radiation",
        "direct_solar_radiation",
        "diffuse_radiation",
        "latitude",
        "longitude",
    ]
    location_cols = ["longitude", "latitude", "county"]
    target_cols = [
        "target",
        "county",
        "is_business",
        "product_type",
        "is_consumption",
        "datetime",
    ]

    def __init__(self):
        self.df_data = pl.read_csv(
            os.path.join(self.root, "train.csv"),
            columns=self.data_cols,
            try_parse_dates=True,
        )
        self.df_client = pl.read_csv(
            os.path.join(self.root, "client.csv"),
            columns=self.client_cols,
            try_parse_dates=True,
        )
        self.df_gas_prices = pl.read_csv(
            os.path.join(self.root, "gas_prices.csv"),
            columns=self.gas_prices_cols,
            try_parse_dates=True,
        )
        self.df_electricity_prices = pl.read_csv(
            os.path.join(self.root, "electricity_prices.csv"),
            columns=self.electricity_prices_cols,
            try_parse_dates=True,
        )
        self.df_electricity_prices = self.df_electricity_prices.with_columns(
            self.df_electricity_prices['euros_per_mwh'].abs().alias('euros_per_mwh')
        )
        self.df_forecast_weather = pl.read_csv(
            os.path.join(self.root, "forecast_weather.csv"),
            columns=self.forecast_weather_cols,
            try_parse_dates=True,
        )
        self.df_historical_weather = pl.read_csv(
            os.path.join(self.root, "historical_weather.csv"),
            columns=self.historical_weather_cols,
            try_parse_dates=True,
        )
        self.df_weather_station_to_county_mapping = pl.read_csv(
            os.path.join(self.root, "weather_station_to_county_mapping.csv"),
            columns=self.location_cols,
            try_parse_dates=True,
        )
        self.df_data = self.df_data.filter(
            pl.col("datetime") >= pd.to_datetime("2022-01-01")
        )
        self.df_target = self.df_data.select(self.target_cols)

        self.schema_data = self.df_data.schema
        self.schema_client = self.df_client.schema
        self.schema_gas_prices = self.df_gas_prices.schema
        self.schema_electricity_prices = self.df_electricity_prices.schema
        self.schema_forecast_weather = self.df_forecast_weather.schema
        self.schema_historical_weather = self.df_historical_weather.schema
        self.schema_target = self.df_target.schema

        self.df_weather_station_to_county_mapping = (
            self.df_weather_station_to_county_mapping.with_columns(
                pl.col("latitude").cast(pl.datatypes.Float32),
                pl.col("longitude").cast(pl.datatypes.Float32),
            )
        )
        
    def run(self):
        self.df_data = self.fill_target(self.df_data)
        self.df_forecast_weather = self.fill_radiation(self.df_forecast_weather)
        self.df_forecast_weather = self.fill_summertime(self.df_forecast_weather)
        self.df_forecast_weather = self.separateTP(self.df_forecast_weather)
        self.df_forecast_weather = self.expForecastHr(self.df_forecast_weather)
        self.df_forecast_weather = self.snow2water(self.df_forecast_weather)
#         self.df_historical_weather = self.histRoll(self.df_historical_weather)
        
        return self
    
    def run_test(self):
        #self.df_data = self.fill_target(self.df_data)
        self.df_forecast_weather = self.separateTP(self.df_forecast_weather)
        self.df_forecast_weather = self.expForecastHr(self.df_forecast_weather)
        self.df_forecast_weather = self.snow2water(self.df_forecast_weather)
#         self.df_historical_weather = self.histRoll(self.df_historical_weather)
        
        return self
    
        
    def fill_target(self, df):
        def _interpolate_group(group):
            group['target'] = group['target'].interpolate(method='linear')
            return group
        return pl.DataFrame(df.to_pandas().groupby(['prediction_unit_id', 'is_consumption']).apply(_interpolate_group))

    def fill_radiation(self, df):
        rad = df.to_pandas()['surface_solar_radiation_downwards'].values
        idx = df['surface_solar_radiation_downwards'].is_null().to_numpy().nonzero()[0]
        for i, ind in enumerate(idx):
            tmp = df[idx][i]
            df_b1 = df.filter(
                pl.col('latitude') == tmp['latitude'], 
                pl.col('longitude') == tmp['longitude'],
                abs(pl.col('forecast_datetime') - tmp['forecast_datetime']) < timedelta(days=2),
                pl.col('forecast_datetime').dt.hour() == tmp['forecast_datetime'].dt.hour(),
                pl.col('hours_ahead') == tmp['hours_ahead'])

            # 결측값 검사 및 안전한 나눗셈
            if df_b1['direct_solar_radiation'][0] is not None and df_b1['surface_solar_radiation_downwards'][0] not in [None, 0] \
               and df_b1['direct_solar_radiation'][2] is not None and df_b1['surface_solar_radiation_downwards'][2] not in [None, 0]:
                fillValue = df_b1['direct_solar_radiation'][1] / ((np.divide(df_b1['direct_solar_radiation'][0], df_b1['surface_solar_radiation_downwards'][0]) +
                    np.divide(df_b1['direct_solar_radiation'][2], df_b1['surface_solar_radiation_downwards'][2]))/2)
                rad[ind] = fillValue
            else:
                rad[ind] = 0  # 결측값 대체를 위한 기본값 설정

        df.replace('surface_solar_radiation_downwards', pl.Series(rad))
        return df


    def fill_summertime(self, df):
        missingDate = list(set(pd.date_range('2021-09-01', '2023-06-02', freq='h')[3:-22]) - set(df.to_pandas()['forecast_datetime'].unique()))
        hrs_ahead = 2
        add_df = pd.DataFrame()
        for date in missingDate:
            tmp = df.filter(abs(pl.col('forecast_datetime') - date) < timedelta(hours=2),
                            pl.col('hours_ahead') <= 2).sort('latitude', 'longitude').to_pandas()
            values_1 = None  # 초기화
            for _, row in tmp.iterrows():
                if row['hours_ahead'] == 1:
                    values_1 = row
                elif row['hours_ahead'] == 2 and values_1 is not None:
                    values_2 = row
                    average_values = pd.Series([(v1+v2)/2 if c != 'forecast_datetime' else date for (v1,v2,c) in zip(values_1,values_2,values_2.keys())],
                                            index=values_2.keys())
                    average_values['hours_ahead'] = hrs_ahead
                    add_df = pd.concat([add_df, average_values.to_frame().T]).reset_index(drop=True)
        return pl.DataFrame(pd.concat([df.to_pandas(), add_df]).reset_index(drop=True))


    def snow2water(self, df): # for historical: [cm]/7->[mm]
        return df.with_columns(
            (df['snowfall']/7).alias('snowfall_mm'))

    def separateTP(self, df): 
    # Adjust the indentation as needed to match the rest of your class
        df = df.with_columns([
            (df['total_precipitation'] - df['snowfall'] / 100).alias('rain')
        ])
        return df

    def expForecastHr(self, df):
    # Ensure the indentation here matches the rest of your class
        def _exp(x):
            return np.exp(x) / np.exp(48)

        df = df.with_columns([
            df['hours_ahead'].apply(_exp).alias('exp_hours_ahead')
        ])
        return df
    
#     def histRoll(self, df): # roll -1hr, for only T & T_d
#         df = df.with_columns([
#             df['temperature'].shift(-1).alias('temperature')
#         ])
#         df = df.with_columns([
#             df['dewpoint'].shift(-1).alias('dewpoint')
#         ])
#         return df

    def update_with_new_data(
        self,
        df_new_client,
        df_new_gas_prices,
        df_new_electricity_prices,
        df_new_forecast_weather,
        df_new_historical_weather,
        df_new_target,
    ):
        
        df_new_client = pl.from_pandas(
            df_new_client[self.client_cols], schema_overrides=self.schema_client
        )
        df_new_gas_prices = pl.from_pandas(
            df_new_gas_prices[self.gas_prices_cols],
            schema_overrides=self.schema_gas_prices,
        )
        df_new_electricity_prices = pl.from_pandas(
            df_new_electricity_prices[self.electricity_prices_cols],
            schema_overrides=self.schema_electricity_prices,
        )
        df_new_forecast_weather = pl.from_pandas(
            df_new_forecast_weather[self.forecast_weather_cols],
            schema_overrides=self.schema_forecast_weather,
        )
        
        df_new_forecast_weather = self.separateTP(df_new_forecast_weather)
        df_new_forecast_weather = self.expForecastHr(df_new_forecast_weather)
        df_new_forecast_weather = self.snow2water(df_new_forecast_weather)

        df_new_historical_weather = pl.from_pandas(
            df_new_historical_weather[self.historical_weather_cols],
            schema_overrides=self.schema_historical_weather,
        )
        df_new_target = pl.from_pandas(
            df_new_target[self.target_cols], schema_overrides=self.schema_target
        )

        self.df_client = pl.concat([self.df_client, df_new_client]).unique(
            ["date", "county", "is_business", "product_type"]
        )
        self.df_gas_prices = pl.concat([self.df_gas_prices, df_new_gas_prices]).unique(
            ["forecast_date"]
        )
        self.df_electricity_prices = pl.concat(
            [self.df_electricity_prices, df_new_electricity_prices]
        ).unique(["forecast_date"])
        self.df_forecast_weather = pl.concat(
            [self.df_forecast_weather, df_new_forecast_weather]
        ).unique(["forecast_datetime", "latitude", "longitude", "hours_ahead"])
        self.df_historical_weather = pl.concat(
            [self.df_historical_weather, df_new_historical_weather]
        ).unique(["datetime", "latitude", "longitude"])
        self.df_target = pl.concat([self.df_target, df_new_target]).unique(
            ["datetime", "county", "is_business", "product_type", "is_consumption"]
        )

    def preprocess_test(self, df_test):
        df_test = df_test.rename(columns={"prediction_datetime": "datetime"})
        df_test = pl.from_pandas(
            df_test[self.data_cols[1:]], schema_overrides=self.schema_data
        )
        return df_test

In [3]:
class FeaturesGenerator:
    """
    This class was copied out from:
    https://www.kaggle.com/code/vitalykudelya/enefit-object-oriented-gbdt
    """
    def __init__(self, data_storage):
        self.data_storage = data_storage

    def _add_general_features(self, df_features):
        df_features = (
            df_features.with_columns(
                pl.col("datetime").dt.ordinal_day().alias("dayofyear"),
                pl.col("datetime").dt.hour().alias("hour"),
                pl.col("datetime").dt.day().alias("day"),
                pl.col("datetime").dt.weekday().alias("weekday"),
                pl.col("datetime").dt.month().alias("month"),
                pl.col("datetime").dt.year().alias("year"),
            )
            .with_columns(
                pl.concat_str(
                    "county",
                    "is_business",
                    "product_type",
                    "is_consumption",
                    separator="_",
                ).alias("segment"),
            )
            .with_columns(
                (np.pi * pl.col("dayofyear") / 183).sin().alias("sin(dayofyear)"),
                (np.pi * pl.col("dayofyear") / 183).cos().alias("cos(dayofyear)"),
                (np.pi * pl.col("hour") / 12).sin().alias("sin(hour)"),
                (np.pi * pl.col("hour") / 12).cos().alias("cos(hour)"),
            )
        )
        return df_features

    def _add_client_features(self, df_features):
        df_client = self.data_storage.df_client

        df_features = df_features.join(
            df_client.with_columns(
                (pl.col("date") + pl.duration(days=2)).cast(pl.Date)
            ),
            on=["county", "is_business", "product_type", "date"],
            how="left",
        )
        return df_features

    def _add_forecast_weather_features(self, df_features):
        df_forecast_weather = self.data_storage.df_forecast_weather
        df_weather_station_to_county_mapping = (
            self.data_storage.df_weather_station_to_county_mapping
        )

        df_forecast_weather = (
            df_forecast_weather.rename({"forecast_datetime": "datetime"})
            #.filter((pl.col("hours_ahead") >= 22) & pl.col("hours_ahead") <= 45)
            .drop("hours_ahead")
            .with_columns(
                pl.col("latitude").cast(pl.datatypes.Float32),
                pl.col("longitude").cast(pl.datatypes.Float32),
            )
            .join(
                df_weather_station_to_county_mapping,
                how="left",
                on=["longitude", "latitude"],
            )
            .drop("longitude", "latitude")
        )

        df_forecast_weather_date = (
            df_forecast_weather.group_by("datetime").mean().drop("county")
        )

        df_forecast_weather_local = (
            df_forecast_weather.filter(pl.col("county").is_not_null())
            .group_by("county", "datetime")
            .mean()
        )
        
        for hours_lag in [0]: # 7 * 24]:
            df_features = df_features.join(
#                 df_forecast_weather_date,
                df_forecast_weather_date.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),
                on="datetime",
                how="left",
                suffix=f"_forecast", #_{hours_lag}h",
            )
            df_features = df_features.join(
#                 df_forecast_weather_local,
                df_forecast_weather_local.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),
                on=["county", "datetime"],
                how="left",
                suffix=f"_forecast_local", #_{hours_lag}h",
            )

        return df_features

    def _add_historical_weather_features(self, df_features):
        df_historical_weather = self.data_storage.df_historical_weather
        df_weather_station_to_county_mapping = (
            self.data_storage.df_weather_station_to_county_mapping
        )

        df_historical_weather = (
            df_historical_weather.with_columns(
                pl.col("latitude").cast(pl.datatypes.Float32),
                pl.col("longitude").cast(pl.datatypes.Float32),
            )
            .join(
                df_weather_station_to_county_mapping,
                how="left",
                on=["longitude", "latitude"],
            )
            .drop("longitude", "latitude")
        )

        df_historical_weather_date = (
            df_historical_weather.group_by("datetime").mean().drop("county")
        )

        df_historical_weather_local = (
            df_historical_weather.filter(pl.col("county").is_not_null())
            .group_by("county", "datetime")
            .mean()
        )


        for hours_lag in [0]: #[2 * 24, 7 * 24]:
            df_features = df_features.join(
                df_historical_weather_date.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),
                on="datetime",
                how="left",
                suffix=f"_historical", #_{hours_lag}h",
            )
            df_features = df_features.join(
                df_historical_weather_local.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),
                on=["county", "datetime"],
                how="left",
                suffix=f"_historical_local", #_{hours_lag}h",
            )

#         for hours_lag in [0]: #[1 * 24]:
#             df_features = df_features.join(
#                 df_historical_weather_date.with_columns(
#                     pl.col("datetime") #+ pl.duration(hours=hours_lag),
#                     pl.col("datetime").dt.hour().alias("hour"),
#                 )
#                 #.filter(pl.col("hour") <= 10)
#                 .drop("hour"),
#                 on="datetime",
#                 how="left",
#                 suffix=f"_historical_{hours_lag}h",
#             )

        return df_features

#     def _add_target_features(self, df_features):
#         df_target = self.data_storage.df_target

#         df_target_all_type_sum = (
#             df_target.group_by(["datetime", "county", "is_business", "is_consumption"])
#             .sum()
#             .drop("product_type")
#         )

#         df_target_all_county_type_sum = (
#             df_target.group_by(["datetime", "is_business", "is_consumption"])
#             .sum()
#             .drop("product_type", "county")
#         )

#         for hours_lag in [
#             2 * 24,
#             3 * 24,
#             4 * 24,
#             5 * 24,
#             6 * 24,
#             7 * 24,
#             8 * 24,
#             9 * 24,
#             10 * 24,
#             11 * 24,
#             12 * 24,
#             13 * 24,
#             14 * 24,
#             6,      ###
#             12,     ###
#             84,     ###
#             3096,   ### add juwon
#         ]:
#             df_features = df_features.join(
#                 df_target.with_columns(
#                     pl.col("datetime") + pl.duration(hours=hours_lag)
#                 ).rename({"target": f"target_{hours_lag}h"}),
#                 on=[
#                     "county",
#                     "is_business",
#                     "product_type",
#                     "is_consumption",
#                     "datetime",
#                 ],
#                 how="left",
#             )

#         for hours_lag in [2 * 24, 3 * 24, 7 * 24, 14 * 24]:
#             df_features = df_features.join(
#                 df_target_all_type_sum.with_columns(
#                     pl.col("datetime") + pl.duration(hours=hours_lag)
#                 ).rename({"target": f"target_all_type_sum_{hours_lag}h"}),
#                 on=["county", "is_business", "is_consumption", "datetime"],
#                 how="left",
#             )

#             df_features = df_features.join(
#                 df_target_all_county_type_sum.with_columns(
#                     pl.col("datetime") + pl.duration(hours=hours_lag)
#                 ).rename({"target": f"target_all_county_type_sum_{hours_lag}h"}),
#                 on=["is_business", "is_consumption", "datetime"],
#                 how="left",
#                 suffix=f"_all_county_type_sum_{hours_lag}h",
#             )

#         cols_for_stats = [
#             f"target_{hours_lag}h" for hours_lag in [2 * 24, 3 * 24, 4 * 24, 5 * 24]
#         ]
#         df_features = df_features.with_columns(
#             df_features.select(cols_for_stats).mean(axis=1).alias(f"target_mean"),
#             df_features.select(cols_for_stats)
#             .transpose()
#             .std()
#             .transpose()
#             .to_series()
#             .alias(f"target_std"),
#         )

#         for target_prefix, lag_nominator, lag_denomonator in [
#             ("target", 24 * 7, 24 * 14),
#             ("target", 24 * 2, 24 * 9),
#             ("target", 24 * 3, 24 * 10),
#             ("target", 24 * 2, 24 * 3),
#             ("target_all_type_sum", 24 * 2, 24 * 3),
#             ("target_all_type_sum", 24 * 7, 24 * 14),
#             ("target_all_county_type_sum", 24 * 2, 24 * 3),
#             ("target_all_county_type_sum", 24 * 7, 24 * 14),
#         ]:
#             df_features = df_features.with_columns(
#                 (
#                     pl.col(f"{target_prefix}_{lag_nominator}h")
#                     / (pl.col(f"{target_prefix}_{lag_denomonator}h") + 1e-3)
#                 ).alias(f"{target_prefix}_ratio_{lag_nominator}_{lag_denomonator}")
#             )

#         return df_features

    def _reduce_memory_usage(self, df_features):
        df_features = df_features.with_columns(pl.col(pl.Float64).cast(pl.Float32))
        return df_features

    def _drop_columns(self, df_features):
        df_features = df_features.drop(
           "datetime", "hour", "dayofyear"
        )
        return df_features

    def _to_pandas(self, df_features, y):
        cat_cols = [
            "county",
            "is_business",
            "product_type",
            "is_consumption",
            "segment",
        ]

        if y is not None:
            df_features = pd.concat([df_features.to_pandas(), y.to_pandas()], axis=1)
        else:
            df_features = df_features.to_pandas()

        df_features = df_features.set_index("row_id")
        df_features[cat_cols] = df_features[cat_cols].astype("category")

        return df_features

    def generate_features(self, df_prediction_items):
        if "target" in df_prediction_items.columns:
            df_prediction_items, y = (
                df_prediction_items.drop("target"),
                df_prediction_items.select("target"),
            )
        else:
            y = None

        df_features = df_prediction_items.with_columns(
            pl.col("datetime").cast(pl.Date).alias("date"),
        ).with_columns(pl.col("datetime").dt.cast_time_unit("us").alias("datetime"))

        for add_features in [
            self._add_general_features,
            self._add_client_features,
            self._add_forecast_weather_features,
            self._add_historical_weather_features,
            #self._add_target_features,
            self._reduce_memory_usage,
            self._drop_columns,
        ]:
            df_features = add_features(df_features)

        df_features = self._to_pandas(df_features, y)

        return df_features

In [4]:
def convert_2_dataloader(df, _seed):

    # define categorical and continous numerical feature column names (on small number of features)
    # from train.csv
    cat_names = ["county", "is_business", "product_type", "is_consumption", "segment"]
    # from datetime column
    cat_names += ["weekday", "month", 'sin(dayofyear)', 'cos(dayofyear)', 'sin(hour)', 'cos(hour)']
    # from https://www.kaggle.com/code/albansteff/enefit-estonian-holidays-lb-65-79 notebook
    cat_names += ['country_holiday']
    
    # from client.csv
    cont_names = ["installed_capacity", "eic_count"]
    
    # from forecast_weather.csv (next 0 hours)
    cont_names += [_ for _ in df.columns if "_forecast_0h" in _]
    cont_names += [_ for _ in df.columns if "_forecast_local_0h" in _]
    # from forecast_weather.csv (next 24 hours)
    cont_names += [_ for _ in df.columns if "_forecast_24h" in _]
    cont_names += [_ for _ in df.columns if "_forecast_local_24h" in _]
    
    # from historical_weather.csv (last 24/48 hours)
    cont_names += [_ for _ in df.columns if "_historical_24h" in _]
    cont_names += [_ for _ in df.columns if "_historical_48h" in _]
    cont_names += [_ for _ in df.columns if "_historical_local_48h" in _]
    
    # add all historical target values (last n hours)
    cont_names += df.filter(regex=("target_.[0-9]*h")).columns.tolist()
    cont_names += ['target_mean', 'target_std']
    
    # added aggregated target values
    cont_names += [_ for _ in df.columns if "target_all_" in _]
    
    # add ratios between last kown target values
    cont_names += df.filter(regex=("target_ratio_.[0-9]")).columns.tolist()
    
    procs = [Categorify, FillMissing, Normalize]
    
    # log transform target variable
    df.loc[:, 'target'] = np.log1p(df['target'])
        
    # convert pandas DataFrame to fastai DataLoader object
    # code snippet taken from
    # https://docs.fast.ai/tabular.learner.html
    splits = RandomSplitter(valid_pct=0.2, seed = _seed)(df)
    
    # tabular object (only categorical features)
    to = TabularPandas(df[cat_names + cont_names + ["target"]],
                       procs = procs,
                       cat_names = cat_names,
                       cont_names = cont_names,
                       y_names = ["target"],
                       splits=splits)
    # create dataloader
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dls = to.dataloaders(BATCH_SIZE, device = device)
     
    # return all dataloaders as tuple
    return dls

def add_custom_features(df):
    """
    Function inspired by notebook:
    https://www.kaggle.com/code/albansteff/enefit-estonian-holidays-lb-65-79
    """
    
    # code bellow same as in NB v17 add_holidays_as_binary_features function
    estonian_holidays = holidays.country_holidays('EE', years=range(2021, 2026))
    estonian_holidays = [pd.to_datetime(_) for _ in estonian_holidays.keys()]
    
    df['country_holiday'] = df['date'].isin(estonian_holidays) * 1
    del df['date']
    
    # log transform histrocial target values
    _cols = df.filter(regex=("target_.[0-9]*h")).columns.tolist()
    for _col in _cols:
        df.loc[:, _col] = np.log1p(df[_col])
    
    # log transform aggregated target values
    _cols = [_ for _ in df.columns if "target_all_" in _]
    for _col in _cols:
        df.loc[:, _col] = np.log1p(df[_col])  
    
    return df

In [5]:
class DataTransformer:
    def __init__(self, df):
        self.df = df

    def transform(self):
        self.add_season()
        self.add_daypart_with_sin_hour()
        self.add_feels_like_temperature()
        #self.add_energy_usage_trend()
        self.add_temp_change()
        self.add_prec_change()
        self.add_autocorr_features()
#         self.add_energy_price_volatility_and_trend()
        #self.perform_clustering()
        self.analyze_transit_and_charging_access()
        return self.df

    def add_season(self):
        def get_season(month):
            if month in [3, 4, 5]:
                return 1 #spring
            elif month in [6, 7, 8]:
                return 2 #summer
            elif month in [9, 10, 11]:
                return 3 #fall
            else:
                return 4 #winter
        
        self.df['season'] = self.df['month'].apply(get_season)

    def add_daypart_with_sin_hour(self):
        def get_daypart(sin_hour):
            if sin_hour > 0:
                return 1  # sin(hour) 양수: 오전~오후
            else:
                return 2      # sin(hour) 음수: 저녁~밤

        self.df['daypart'] = self.df['sin(hour)'].apply(get_daypart)

    def add_feels_like_temperature(self):
        def calculate_feels_like(T, u, v):
            wind_speed = (u**2 + v**2)**0.5
            if wind_speed < 4.8:
                return T
            else:
                return 13.12 + 0.6215 * T - 11.37 * (wind_speed ** 0.16) + 0.3965 * T * (wind_speed ** 0.16)

        self.df['feels_like_temp'] = self.df.apply(lambda row: calculate_feels_like(row['temperature'], row['10_metre_u_wind_component'], row['10_metre_v_wind_component']), axis=1)


#     def add_energy_usage_trend(self, period = 7):
#         self.df['energy_trend'] = self.df['target'].rolling(window = period).mean()

    ##### need to fix --> filter unique unit
    def add_temp_change(self, interval = 24):
        self.df['temp_change'] = self.df['temperature'].diff(periods = interval)
        
    ##### need to fix --> filter unique unit
    def add_prec_change(self):
        self.df['precipitation_change'] = self.df['total_precipitation'].diff()

    def add_autocorr_features(self, lags = 10):
        acf_values = acf(self.df['target'], nlags = lags)
        pacf_values = pacf(self.df['target'], nlags = lags)
        for i in range(lags+1):
            self.df[f'acf_lag_{i}'] = acf_values[i]
            self.df[f'pacf_lag{i}'] = pacf_values[i]

#     def add_energy_price_volatility_and_trend(self, window = 7):
#         self.df['energy_price_volatility'] = self.df['target'].rolling(window = window).std()

    # def perform_clustering(self, n_clusters = 3, features = None):
    #     if features is None:
    #         features = ['target_24h', 'target_48h', 'temperature', 'cloudcover_total']
    #     scaler = StandardScaler()
    #     scaled_data = scaler.fit_transform(self.df[features])

    #     kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    #     self.df['cluster'] = kmeans.fit_predict(scaled_data)

    def analyze_transit_and_charging_access(self):

        results = []
        for consumption_status in [0, 1]:
            subset = self.df[self.df['is_consumption'] == consumption_status]

            # 대중교통 이용률 분석
            business_hours_energy = subset[subset['is_business'] == 1]['target'].mean()
            non_business_hours_energy = subset[subset['is_business'] == 0]['target'].mean()
            transit_usage_estimate = business_hours_energy - non_business_hours_energy

            # 전기차 충전소 접근성 분석
            high_capacity_energy = subset[subset['installed_capacity'] > subset['installed_capacity'].median()]['target'].mean()
            low_capacity_energy = subset[subset['installed_capacity'] <= subset['installed_capacity'].median()]['target'].mean()
            charging_access_estimate = high_capacity_energy - low_capacity_energy

            results.append((consumption_status, transit_usage_estimate, charging_access_estimate))

        # 결과를 하나의 컬럼으로 합침
        for consumption_status, transit_estimate, charging_estimate in results:
            self.df[f'transit_usage_estimate_{consumption_status}'] = transit_estimate
            self.df[f'charging_access_estimate_{consumption_status}'] = charging_estimate

In [6]:
#train_dataset data 변환 (weekday -> weekend, wind dir, speed -> U10, V10)
class TrainDataTransform:
    def __init__(self, df):
        self.df = df

    def transform(self):
        self.is_weekend()
        self.wind_data_to_UV()
        return self.df

    #weekend 판별 함수
    def is_weekend(self):
        self.df['is_weekend'] = np.where(self.df['weekday'] > 4, 1, 0)
        
    def wind_data_to_UV(self):
        self.df['U10'] = self.df['windspeed_10m'] * np.cos(np.radians(270 - self.df['winddirection_10m']))
        self.df['V10'] = self.df['windspeed_10m'] * np.sin(np.radians(270 - self.df['winddirection_10m']))
        

In [7]:
#     df_test = data_storage.preprocess_test(df_test)
#     df_test = data_storage.run()
    
#     features_generator = FeaturesGenerator(data_storage = df_test)
#     df_test = features_generator.generate_features(df_test)

data_storage = DataStorage()
data_storage = data_storage.run()

features_generator = FeaturesGenerator(data_storage=data_storage)

train_dataset = features_generator.generate_features(data_storage.df_data)
# exclude rows with missing target value
train_dataset = train_dataset[train_dataset['target'].notnull()]
# add estonian holidays
train_dataset = add_custom_features(train_dataset)

In [8]:
train_dataset.head()
#### historical, forecast 말고 걍 날씨column은 어디코드부분에서 추가된거?

,county,is_business,product_type,is_consumption,prediction_unit_id,day,weekday,month,year,segment,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour),eic_count,installed_capacity,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation,rain,exp_hours_ahead,snowfall_mm,temperature_forecast_local,dewpoint_forecast_local,cloudcover_high_forecast_local,cloudcover_low_forecast_local,cloudcover_mid_forecast_local,cloudcover_total_forecast_local,10_metre_u_wind_component_forecast_local,10_metre_v_wind_component_forecast_local,direct_solar_radiation_forecast_local,surface_solar_radiation_downwards_forecast_local,snowfall_forecast_local,total_precipitation_forecast_local,rain_forecast_local,exp_hours_ahead_forecast_local,snowfall_mm_forecast_local,temperature_historical,dewpoint_historical,rain_historical,snowfall_historical,surface_pressure,cloudcover_total_historical,cloudcover_low_historical,cloudcover_mid_historical,cloudcover_high_historical,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation_historical,diffuse_radiation,temperature_historical_local,dewpoint_historical_local,rain_historical_local,snowfall_historical_local,surface_pressure_historical_local,cloudcover_total_historical_local,cloudcover_low_historical_local,cloudcover_mid_historical_local,cloudcover_high_historical_local,windspeed_10m_historical_local,winddirection_10m_historical_local,shortwave_radiation_historical_local,direct_solar_radiation_historical_local,diffuse_radiation_historical_local,target,country_holiday
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
366048,0,0,1,0,0,1,6,1,2022,0_0_1_0,0.017166,0.999853,0.000000,1.000000,148.0,1345.689941,-0.246805,-0.777223,0.367615,0.930420,0.591207,0.995787,0.617221,-0.222493,0.0,0.0,0.000016,0.000065,0.000065,1.839397e-01,0.000002,-4.124589,-4.592993,0.749156,0.923236,0.957959,0.999995,0.002135,0.237211,0.0,0.0,8.940697e-07,7.763738e-07,7.674331e-07,1.839397e-01,1.277242e-07,0.354464,-0.122321,0.047321,0.006250,997.277649,96.589287,84.080360,45.553570,25.169643,2.248512,238.607147,5.669643,1.821429,3.848214,-2.483333,-2.900000,0.033333,0.035000,996.099976,100.0,79.666664,82.666664,18.833334,1.625000,253.833328,0.0,0.0,0.0,0.0,1
366178,0,0,1,0,0,1,6,1,2022,0_0_1_0,0.017166,0.999853,0.258819,0.965926,148.0,1345.689941,-0.313706,-0.812520,0.374891,0.954259,0.650327,0.999100,0.330942,-0.536234,0.0,0.0,0.000015,0.000081,0.000081,5.000000e-01,0.000002,-3.848140,-4.455989,0.529856,0.974208,0.996208,1.000000,-0.267968,-0.146743,0.0,0.0,3.735224e-06,3.789857e-06,3.752505e-06,5.000000e-01,5.336035e-07,0.418750,-0.043750,0.082143,0.012500,997.504456,95.071426,87.616074,52.464287,23.544643,2.388145,242.080353,4.875000,1.357143,3.517857,-1.983333,-2.333333,0.083333,0.093333,996.183350,100.0,84.666664,92.833336,64.000000,1.439815,247.333328,0.0,0.0,0.0,0.0,1
366308,0,0,1,0,0,1,6,1,2022,0_0_1_0,0.017166,0.999853,0.500000,0.866025,148.0,1345.689941,0.239324,-0.170470,0.499605,0.993014,0.763934,0.999728,1.173773,-1.291043,0.0,0.0,0.000050,0.000084,0.000084,5.130940e-11,0.000007,-1.917253,-2.317501,0.612394,0.998734,1.000000,1.000000,-0.589855,-1.035992,0.0,0.0,4.971027e-05,4.986400e-05,4.936690e-05,5.130940e-11,7.101468e-06,0.412500,0.015179,0.104464,0.035000,997.558899,98.035713,92.919640,68.116074,35.767857,7.843998,210.821426,4.232143,2.008929,2.223214,-1.366667,-1.750000,0.083333,0.140000,996.200012,100.0,93.500000,99.666664,84.333336,8.106482,211.666672,0.0,0.0,0.0,0.0,1
366438,0,0,1,0,0,1,6,1,2022,0_0_1_0,0.017166,0.999853,0.707107,0.707107,148.0,1345.689941,0.205129,-0.233177,0.476371,0.985521,0.834310,0.999999,1.033344,-1.940564,0.0,0.0,0.000208,0.000330,0.000328,1.394734e-10,0.000030,-1.508327,-1.869747,0.700434,0.997820,1.000000,1.000000,-1.024466,-1.918736,0.0,0.0,5.386000e-04,5.460407e-04,5.406546e-04,1.

In [9]:
#이거 먼저 와야함, 예림 추가
train_dataset['eic_count'] = train_dataset['eic_count'].fillna(method = 'bfill')
train_dataset['installed_capacity'] = train_dataset['installed_capacity'].fillna(method = 'bfill')

######################## 이부분에서 missing value확인하기?

#add yelim
train_dataset = DataTransformer(train_dataset)
train_dataset = train_dataset.transform()

#add joonyong
train_dataset = TrainDataTransform(train_dataset)
train_dataset = train_dataset.transform()

### drop county == 12 , prec_change Nan,  yelim added ####
train_dataset = train_dataset[train_dataset['county'] != 12]
train_dataset['precipitation_change'] = train_dataset['precipitation_change'].fillna(method='bfill')

In [10]:
### 2023/05/30 10am 이후로 weather 없음 --> drop
train_dataset = train_dataset[~train_dataset['rain_historical'].isna()]

In [11]:
### temp_change 24개 window 채우기
'''
to do
'''

### 임시 코드
train_dataset = train_dataset[~train_dataset['temp_change'].isna()]

In [12]:
for c in train_dataset.columns:
    if train_dataset[c].isna().sum() > 0:
        print(c, train_dataset[c].isna().sum())

In [13]:
import enefit
from sklearn.linear_model import LinearRegression

# 모델 초기화
simple_model = LinearRegression()

train_dataset = train_dataset.fillna(method='bfill')
train_dataset = train_dataset.fillna(method='ffill')
train_dataset = pd.get_dummies(train_dataset, drop_first=True)
# 데이터 준비 및 모델 학습
# df_train_features: 훈련 데이터프레임
X_train = train_dataset.drop(columns=['target'])
y_train = train_dataset['target']
simple_model.fit(X_train, y_train)


LinearRegression()

In [14]:
import enefit

env = enefit.make_env()
iter_test = env.iter_test()

In [15]:
%%time
for (
    df_test, 
    df_new_target, 
    df_new_client, 
    df_new_historical_weather,
    df_new_forecast_weather, 
    df_new_electricity_prices, 
    df_new_gas_prices, 
    df_sample_prediction
) in iter_test:

    data_storage.update_with_new_data(
        df_new_client=df_new_client,
        df_new_gas_prices=df_new_gas_prices,
        df_new_electricity_prices=df_new_electricity_prices,
        df_new_forecast_weather=df_new_forecast_weather,
        df_new_historical_weather=df_new_historical_weather,
        df_new_target=df_new_target
    )
    df_test = data_storage.preprocess_test(df_test)
    df_test = data_storage.run_test()
    
    df_test = features_generator.generate_features(df_test.df_data)
    df_test = df_test[df_test['target'].notnull()]
    df_test = add_custom_features(df_test)

    #이거 먼저 와야함, 예림 추가
    df_test['eic_count'] = df_test['eic_count'].fillna(method = 'bfill')
    df_test['installed_capacity'] = df_test['installed_capacity'].fillna(method = 'bfill')
    #add yelim
    df_test = DataTransformer(df_test)
    df_test = df_test.transform()
    #add joonyong
    df_test = TrainDataTransform(df_test)
    df_test = df_test.transform()
    ### drop county == 12 , prec_change Nan,  yelim added ####
    df_test = df_test[df_test['county'] != 12]
    df_test_features = df_test.fillna(method='bfill')
    df_test_features = df_test_features.fillna(method='ffill')
    df_test_features = df_test_features.drop(columns=['target'])
    df_test_features = pd.get_dummies(df_test_features, drop_first=True)
    print(df_test_features.shape)
    
    common_row_ids = df_test_features.index.intersection(df_sample_prediction['row_id'])
    
    for row_id in common_row_ids:
        data_row = df_test_features.loc[row_id]
        prediction = simple_model.predict(data_row.values.reshape(1,-1))
        df_sample_prediction.loc[df_sample_prediction['row_id'] == row_id, 'target'] = prediction[0]

    env.predict(df_sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
(1627536, 261)
(1627536, 261)
(1627536, 261)
(1627536, 261)
CPU times: user 5min 36s, sys: 36.5 s, total: 6min 13s
Wall time: 5min 37s


In [16]:
df_test_features = pd.get_dummies(df_test_features, drop_first=True)
print(df_test_features.dtypes)

prediction_unit_id                                    int64
day                                                    int8
weekday                                                int8
month                                                  int8
year                                                  int32
sin(dayofyear)                                      float32
cos(dayofyear)                                      float32
sin(hour)                                           float32
cos(hour)                                           float32
eic_count                                           float64
installed_capacity                                  float32
temperature                                         float32
dewpoint                                            float32
cloudcover_high                                     float32
cloudcover_low                                      float32
cloudcover_mid                                      float32
cloudcover_total                        

In [ ]:
X_train.shape

In [17]:
df_sample_prediction.head()

,row_id,target
0,2015232,341.638887
1,2015233,568.344305
2,2015234,-93.139414
3,2015235,-79.937082
4,2015236,1351.001606
